# A Hypergraph Analysis of the novel Les Miserables  by Victor Hugo  

<img src="../images/LesMisStudyOpeningGraphic.png" >

As readers, we are mesmerized by the prose and drama and are horrified by the meanness of the time. But as mathematicians, we delight in the opportunity to study the relationships among the characters and discover what the hypergraphs they generate tell us about the story. We use the data available from the [Stanford GraphBase]( https://www-cs-faculty.stanford.edu/~knuth/sgb.html).

In [ ]:
import pandas as pd
import itertools as itt
from collections import defaultdict
import matplotlib.pyplot as plt

try:
    import hypernetx as hnx
except ImportError:
    print("Installing HyperNetX.........")
    !pip install hypernetx --quiet 2> /dev/null
    print("Installation complete; please rerun this cell in order for the rest of the cells to use HyperNetX.")
    exit()

import shutup
shutup.mute_warnings()

In [ ]:
lm = hnx.LesMis()

### Basic Structure of the Novel
The novel is broken into five parts, which we will here reference as volumes: **Fantine**, **Cosette**, **Marius**, **St. Denis**, and **Jean Valjean**. Each volume is subdivided into books, each book into chapters, and each chapter into scenes. By shifting the level of subdivision, we are able to construct multiple hypergraphs modeling character interactions and relationships.

In [ ]:
volumes = lm.volumes
volumes

In [ ]:
### List of Characters
names = lm.df_names.set_index('Symbol')
names

In [ ]:
### Who are the characters?
lm.df_names

In [ ]:
### List of Characters as they appear in each scene
scenes = lm.df_scenes;scenes

In [ ]:
h = hnx.Hypergraph(scenes,
                  edge_col='Volume',
                  node_col = 'Characters',
                  node_properties = lm.df_names)

In [ ]:
h.properties

## Hypergraph with each Volume as an Edge

We first generate a hypergraph studying the relationship between volumes.  The edges are the volumes and the characters who appear in each volume are the nodes. 

In [ ]:
### Construct the edges as a dictionary named by the name of the Volume
volume_edges = {v : set(scenes.loc[scenes.Volume == v]['Characters']) for v in range(1, 6)}

### Construct a hypergraph made up of volume_edges
HV = hnx.Hypergraph(volume_edges,name='Volumes')
for node in HV.nodes:
    HV.nodes[node].name = names.loc[node]['FullName']
    HV.nodes[node].description = names.loc[node]['Description']

In [ ]:
## What is in an edge?
HV.edges[1]

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(HV)

In [ ]:
### Collapse the nodes to understand the relationships (same as diagram in title)
plt.figure(figsize=[10,10])
hnx.draw(HV.collapse_nodes(),with_node_counts=True)

Noting the characters in the intersections between edges gives a sense of their importance. Which characters are central to the novel?

In [ ]:
### Who are the four characters belonging to all five volumes?
volume_char_sets = list(set(HV.edges[idx]) for idx in range(1,6))
core_characters = list(set.intersection(*volume_char_sets))
names.loc[core_characters]

In [ ]:
### Who are the characters belonging to the different intersections? 
## Replace the values in the array with the volumes of interest.
vols = [3,4]
volume_char_sets = list(set(HV.edges[idx]) for idx in vols)
chars_of_interest = list(set.intersection(*volume_char_sets))
names.loc[chars_of_interest]


In [ ]:
## What are the intersection sizes:
volume_intersections = dict()
for pair in itt.combinations(range(1,6),2):
    volume_char_sets = list(set(HV.edges[idx]) for idx in pair)
    titlepair = (volumes.title[pair[0]],volumes.title[pair[1]])
    volume_intersections[tuple(pair)] = len(set.intersection(*volume_char_sets))
print("Number of Characters shared by pairs of volumes:")
pd.DataFrame.from_dict(volume_intersections,orient="index")

### Highlights from each Volume (A very very very short description...)

Fantine: Volume 1 Lays the foundation for the novel. Most of the characters do not appear in subsequent volumes. Most important of these is Fantine. As mother of Cosette, she sacrifices her life in the support of her daughter and lays the charge on Jean Valjean to care for Cosette when she dies.  In contrast to Fantine, Jean Valjean and Cosette appear in all the volumes. A central story to the novel follows their travels as they flee from the relentless pursuit by Javert and the dogged and often comical abuses of Thenardier. Jean Valjean, originally convicted for stealing bread, begins as a hardened convict but through the mercy of a bishop is transformed into a philanthropist.

Cosette: Volume 2 follows Cosette's liberation from her caretakers, the Thenardiers, by Jean Valjean. They flee into hiding from Javert and find refuge in a convent, where Valjean works as a gardener and Cosette is educated. Much character development is done, including a long description of Waterloo ending with the singular way in which M. Thenardier obtained a silver cross of the Legion of Honour while saving the life of one Pontmercy.

Marius: Marius Pontmercy, son of an officer in Napoleon's army and grandson of a Royalist, experiences conflicting loyalties and ultimately turns his back on friends and family and lives among the poor. He sees and eventually falls in love with Cosette. In honor of his father, he attempts to help M. Thenardier but discovers his treachery when M. Thenardier attempts to murder the person he takes to be Cosette's father.

St. Denis: With his love for Cosette thwarted, Marius joins a group of students to participate in an uprising known as the June rebellion. They construct a barricade near the Rue Saint-Denis.

Jean Valjean: Jean Valjean saves Marius's life when soldiers overwhelm the barricades. Marius discovers Jean Valjean's true identity from Thenardier. Jean Valjean dies at peace with Cosette and Marius.



## Hypergraph of each Volume using Books as edges. 


In [ ]:
### Construct a hypergraph for each volume
### The edges will be the books and the nodes the characters.
fantine = scenes.loc[scenes.Volume == 1]
cosette = scenes.loc[scenes.Volume == 2]
marius = scenes.loc[scenes.Volume == 3]
stdenis = scenes.loc[scenes.Volume == 4]
jeanvaljean = scenes.loc[scenes.Volume == 5]

In [ ]:
### Construct the edges as a dictionaries
vols = [0,fantine, cosette, marius, stdenis, jeanvaljean]
HB = dict()
for idx in range(1,6):
    book_edges = dict()
    for book in vols[idx].Book:
        book_edges[book] = set(vols[idx].loc[vols[idx].Book == book]['Characters'])
    ### Construct a hypergraph made up of volume_edges
    HB[idx] = hnx.Hypergraph(book_edges,name=f"{volumes.title[idx]}-Books")
    for node in HB[idx].nodes:
        HB[idx].nodes[node].name = names.loc[node]['FullName']
        HB[idx].nodes[node].description = names.loc[node]['Description']

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(HB[1].collapse_nodes(),with_node_counts=True)

In [ ]:
### In the final book there is a long section on the Paris Sewers. 
### The only character in that section was the creator of the sewers.
plt.figure(figsize=[10,10])
hnx.draw(HB[5])

In [ ]:
HB[5].nodes['BS']

In [ ]:
HB[5].nodes['BS'].memberships ## Bruneseau belongs to the 2nd book in volume 5

In [ ]:
### Separate the components and consider the large one
c1,c2 = list(HB[5].component_subgraphs(return_singletons=True))
hnx.draw(c1)

In [ ]:
### Note in the first book of this volume the soldiers confront the students at the barricades
### Most of the students are killed
### In the 4th book Javert dies alone
### At the center, in the last scene (8), there is only JV and CO
plt.figure(figsize=[10,10])
hnx.draw(c1)

In [ ]:
HB[5].edges[8]

## Who knew who?

We can examine the interactions between certain characters by studying their neighborhoods and induced subgraphs.


In [ ]:
## In Volume 1, Restrict to the nodes in a neighborhood of Fantine 
## and construct the subhypergraph of HB[1] restricted to these nodes 

FNnodes = list(HB[1]['FN'])
FNnodes.append('FN')
FNNeighborhood = HB[1].restrict_to_nodes(FNnodes)

In [ ]:
FNNeighborhood.incidence_dict

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(FNNeighborhood)

In [ ]:
## In Volume 1, Restrict to a neighborhood of Jean Valjean
JVnodes = list(HB[1]['JV'])
JVnodes.append('JV')
JVNeighborhood = HB[1].restrict_to_nodes(JVnodes)
JVNeighborhood.incidence_dict

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(JVNeighborhood)

In [ ]:
## Combine the subgraphs
Fantine_edges = list(FNNeighborhood.edges.elements.values())
JVFNHypergraph = HB[1].restrict_to_nodes(JVnodes)

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(JVFNHypergraph)

## Hypergraph of each Volume using scenes as edges

In [ ]:
## Scene hypergraphs for each Volume indexed by Book,Chapter,Scene.
vols = [0,fantine, cosette, marius, stdenis, jeanvaljean]
scene_edges = defaultdict(list)
HS = dict()
for idx in range(1,6):
    scene_edges = defaultdict(list)
    for row in vols[idx].itertuples():
        scene_edges[f"{row.Book}.{row.Chapter}.{row.Scene}"].append(row.Characters)
    ### Construct a hypergraph made up of scene_edges
    HS[idx] = hnx.Hypergraph(scene_edges,name=f"{volumes.title[idx]}-Scenes")
    for node in HS[idx].nodes:
        HS[idx].nodes[node].name = names.loc[node]['FullName']
        HS[idx].nodes[node].description = names.loc[node]['Description']    

In [ ]:
HS[1].edges.incidence_dict

## Who knew who by Scenes?

### Difference in relationships when drilling down into hierarchy of sets

We consider the neighborhoods of Fantine and Jean Valjean in the Scenes hypergraph. 
Starting with the subgraph generated by the neighbors of Fantine, we restrict to neighbors of Jean Valjean.

In [ ]:
## In Volume 1, Restrict to a neighborhood of Fantine
FNnodes = list(HS[1]['FN'])
FNnodes.append('FN')
FNNeighborhood = HS[1].restrict_to_nodes(FNnodes)
jvfn = list(FNNeighborhood['JV'])
jvfn.append('JV')
FNJVNeighborhood = FNNeighborhood.restrict_to_nodes(jvfn)

In [ ]:
plt.figure(figsize=[25,15])
hnx.draw(FNJVNeighborhood)

While the graph shows FN and JV as central points we can more clearly visualize the core relationships by collapsing the edges

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(FNJVNeighborhood.collapse_edges(),with_edge_labels=False)

While the Books Hypergraph indicated many relationships shared by FN and JV it does not indicate if FN and JV actually encountered each other alone or who they were with. The Scenes Hypergraph drills down in the hierarchy of relationships to those core scenes where actual meetings took place and who else was involved.

## Hypergraph of Scenes in Volume 1 Book 1

In [ ]:
df = scenes.loc[scenes.Volume==1].loc[scenes.Book == 1]
edges = defaultdict(list)
for row in df.itertuples():
    edges[row.Chapter].append(row.Characters)
Hdf = hnx.Hypergraph(edges)

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(Hdf)

In [ ]:
plt.figure(figsize=[10,10])
hnx.draw(Hdf.collapse_nodes_and_edges(),with_node_counts=True,with_edge_counts=True)